# HowTo -PointCloud- From carved volume to point cloud

In this notebook you will learn:

1. how to connect to a ROMI database
2. how to transform the carved volume into point cloud
3. how to visualize the reconstructed point cloud

This notebook **assume** that you have:
- declared the `ROMI_DB` environment variable as the path to the database directory to use
- processed the test dataset with the _geometric pipeline,_ so we can access the fileset containing the data we want to start with...

Remember, the aim of this notebook is to show you how it works "under the hood".
This is not how you should process your data, that is done thanks to the `romi_run_task` CLI tool.

In [ ]:
import os

import ipywidgets as widgets
import numpy as np
from plant3dvision.proc3d import vol2pcd
from plant3dvision.visu import plotly_volume_slicer, plotly_pointcloud
from plantdb import FSDB
from plantdb.io import read_volume

## Connect to the database & get the initial data

If you did not declare a `ROMI_DB` environment variable, you can do it by uncommenting the next cell and setting it to the right value.

In [ ]:
# os.environ['ROMI_DB'] = "/path/to/test/data"

### Connect to the database

In [ ]:
db = FSDB(os.environ['ROMI_DB'])  # requires definition of this environment variable!
db.connect()

Once you are connected to the database, you can list the available scan *dataset* with `db.list_scans()`.

### Select a dataset

We now select a dataset (with the `Dropdown` widget) for the demo:

In [ ]:
scan_name = widgets.Dropdown(options=db.list_scans(), value=db.list_scans()[0], description='Dataset:')
display(scan_name)

In [ ]:
scan = db.get_scan(scan_name.value)

If you did not process this dataset yet, from the `plant3dvision` root directory, you can do it with:
```
romi_run_task AnglesAndInternodes $ROMI_DB/<selected_dataset> --config plant-3d-vision/configs/geom_pipe_real.toml
```

To list the available *filesets* in this *scan dataset*:

In [ ]:
scan.list_filesets()

### Get the Voxel volume fileset

The voxel volume resulting from a _Voxels_ task are to be found in the 'Voxels*' fileset.

In [ ]:
volume_fs = scan.get_filesets(query={"task_name": "Voxels"})[0]
print(volume_fs.path().stem)

### Get and load the carved volume

Once you have access to the right fileset, you may access the volume file and load it as follows:

In [ ]:
volume_file = volume_fs.get_files()[0]
vol = read_volume(volume_file)

### Visualize the carved volume

It is possible to visualize the carved volume with the `plotly_volume_slicer` from `plant3dvision.visu` as follows:

In [ ]:
fig = plotly_volume_slicer(vol)
fig.show()

## Compute the point cloud

We will now convert the carved volume into a point cloud describing the envelope of the object.

### Get the parameters from the volume file metadata

We start by getting the voxel-size and origin of the volume to **correctly place the point cloud in space** and **scale it**.

In [ ]:
vxs = volume_file.get_metadata('voxel_size')
print(f"The volume voxel-size is: {vxs}x{vxs}x{vxs}mm.")
ori = np.array(volume_file.get_metadata('origin'))
print(f"The volume origin is located at: {', '.join(map(str, ori))} (XYZ).")

### Estimate the point cloud

In this step, we will detect the points describing the volume envelope (shell) and compute the normals.

**WARNING**: this involves heavy computation and might take some times depending on your machine.

In [ ]:
pcd = vol2pcd(vol, origin=ori, voxel_size=vxs)

### Visualize the estimated point cloud

Once the point cloud has been obtained, we can visualize it with the `plotly_pointcloud` method from `plant3dvision.visu` as follows:

In [ ]:
fig = plotly_pointcloud(pcd, n_pts=9000)

In [ ]:
fig.show()

Note that this is not EXACTLY the point cloud you estimated, but **a down-sampled version** to keep a fluid viewer.
The number of points kept for this figure is given byt the `n_pts` parameter of the `plotly_pointcloud` method.

We may now **disconnect** from the database as we will not need it anymore:

In [ ]:
db.disconnect()